In [0]:
import json

# Retrieve the bronze_params directly as a widget
bronze_params = dbutils.widgets.get("bronze_params")
print(f"Raw bronze_params: {bronze_params}")

# Parse the JSON string
output_data = json.loads(bronze_params)

# Access individual variables
start_date = output_data.get("start_date", "")
end_date = output_data.get("end_date", "")
bronze_adls = output_data.get("bronze_adls", "")
silver_adls = output_data.get("silver_adls", "")
gold_adls = output_data.get("gold_adls", "")

print(f"Start Date: {start_date}, Bronze ADLS: {bronze_adls}")
     

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-4938722733014854>, line 4
      1 import json
      3 # Retrieve the bronze_params directly as a widget
----> 4 bronze_params = dbutils.widgets.get("bronze_params")
      5 print(f"Raw bronze_params: {bronze_params}")
      7 # Parse the JSON string

File /databricks/python_shell/lib/dbruntime/WidgetHandlerImpl.py:77, in WidgetsHandlerImpl.get(self, name)
     37 def get(self, name):
     38     """ Returns the current value of a widget with the given name.
     39 
     40     :param name: Name of the argument to be accessed
   (...)
     75         ```
     76     """
---> 77     return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, *args)
   1349 command = proto.CALL_COMMAND

In [0]:

from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:

# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

In [0]:
df.head()

Row(geometry=Row(coordinates=[128.5685, 25.9909, 10.0], type='Point'), id='us7000ppcu', properties=Row(alert=None, cdi=2.0, code='7000ppcu', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000ppcu&format=geojson', dmin=0.882, felt=2, gap=64.0, ids=',us7000ppcu,usauto7000ppcu,', mag=5.0, magType='mww', mmi=None, net='us', nst=58, place='79 km ESE of Katsuren-haebaru, Japan', rms=0.42, sig=385, sources=',us,usauto,', status='reviewed', time=1743551992032, title='M 5.0 - 79 km ESE of Katsuren-haebaru, Japan', tsunami=0, type='earthquake', types=',dyfi,internal-moment-tensor,origin,phase-data,', tz=None, updated=1743565691040, url='https://earthquake.usgs.gov/earthquakes/eventpage/us7000ppcu'), type='Feature')

In [0]:
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [0]:
df.head()

Row(id='us7000ppcu', longitude=128.5685, latitude=25.9909, elevation=10.0, title='M 5.0 - 79 km ESE of Katsuren-haebaru, Japan', place_description='79 km ESE of Katsuren-haebaru, Japan', sig=385, mag=5.0, magType='mww', time=1743551992032, updated=1743565691040)

In [0]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)
     

In [0]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)
     

In [0]:
df.head()

Row(id='us7000ppcu', longitude=128.5685, latitude=25.9909, elevation=10.0, title='M 5.0 - 79 km ESE of Katsuren-haebaru, Japan', place_description='79 km ESE of Katsuren-haebaru, Japan', sig=385, mag=5.0, magType='mww', time=datetime.datetime(2025, 4, 1, 23, 59, 52, 32000), updated=datetime.datetime(2025, 4, 2, 3, 48, 11, 40000))

In [0]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"
     

In [0]:
# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)

In [0]:
dbutils.notebook.exit(silver_output_path)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5017597593807953>, line 1
----> 1 dbutils.notebook.exit(silver_output_path)

NameError: name 'silver_output_path' is not defined